In [ ]:
from dkube.sdk import *
import os

In [ ]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
user = os.getenv('USERNAME')
code_name = "tmdb"
train_fs = "tmdb-train-fs"
test_fs  = "tmdb-test-fs"
model_name = "tmdb-model"

In [ ]:
api = DkubeApi(token=existing_token)
print(f"Adding model {model_name}")
model = DkubeModel(user, name=model_name)
try:
    model.update_model_source(source='dvs')
    api.create_model(model)
except BaseException as e:
    print(e)

In [ ]:
api = DkubeApi(token=existing_token)

In [ ]:
import os, json
import kfp
import kfp.compiler as compiler
import random
import string

generate = lambda hint: "{}-{}".format(hint, ''.join([random.choice(string.digits) for n in range(4)]))
runid = 1

In [ ]:
client = kfp.Client(existing_token=existing_token)

In [ ]:
image = "docker.io/ocdr/d3-datascience-sklearn:v0.23.2-1"
fs_inp_path = "/opt/dkube/input"
model_out_path = "/opt/dkube/output"
model_inp_path = "/opt/dkube/model"
train_script = f"cd training; python training.py --fs {train_fs}"
eval_script = f"cd training; python evaluation.py --fs {test_fs}"
transformer_script = "training/transformer.py"
serving_image = "ocdr/sklearnserver:0.23.2"
framework = "sklearn"
f_version = "0.23.2"

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-train-pl',
    description='example training pipeline to submit to leaderboard'
)
def training_pipeline(token):

    train = dkube_training_op(token, json.dumps({"image": image}),
                              framework=framework, version=f_version,
                              program=code_name, run_script=train_script,
                              featuresets=json.dumps([str(train_fs)]), outputs=json.dumps([str(model_name)]),
                              input_featureset_mounts=json.dumps([fs_inp_path]),
                              output_mounts=json.dumps([model_out_path])).set_display_name("Training")
    
    evalt = dkube_training_op(token, json.dumps({"image": image}),
                              framework=framework, version=f_version,
                              program=code_name, run_script=eval_script,
                              featuresets=json.dumps([str(test_fs)]),
                              input_featureset_mounts=json.dumps([fs_inp_path]),
                              models=json.dumps([str(model_name)]),
                              input_model_mounts=json.dumps([model_inp_path])).after(train).set_display_name("Evaluation")
    
    submit = dkube_serving_op(token, train.outputs['artifact'],
                              device="cpu", serving_image=json.dumps({"image": serving_image}),
                              transformer_image=json.dumps({"image": image}),
                              transformer_project=code_name,
                              transformer_code=transformer_script).after(evalt).set_display_name("Serving")

In [ ]:
experiment_name = 'Dkube- Train pl'
train_experiment = client.create_experiment(name=experiment_name)

In [ ]:
import kfp.compiler as compiler
compiler.Compiler().compile(training_pipeline, "train-pipeline.zip")
try:
    pipeline = client.upload_pipeline("train-pipeline.zip", pipeline_name = "Train-pipeline")
except BaseException as e:
    print(e)
runid = 1

In [ ]:
arguments = {"token":existing_token}
run = client.run_pipeline(train_experiment.id, job_name=f"[{pipeline.name}] Run" + str(runid), pipeline_id=pipeline.id, params=arguments)
runid += 1